# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.1.68/12159/1 Dashboard: http://10.20.1.68:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1015,Yvonne,-0.17269366863437452,0.6182017305076226
2000-01-01 00:00:01,972,Ingrid,0.0770378415139592,-0.4569929376888884
2000-01-01 00:00:02,969,Zelda,0.7526081188856284,-0.7897732681216607
2000-01-01 00:00:03,989,George,-0.8309264473349212,0.8385348892030946
2000-01-01 00:00:04,1016,Tim,-0.9643002792571187,0.19544658060732378
2000-01-01 00:00:05,977,Wendy,-0.7830091326178525,0.025288374054839968
2000-01-01 00:00:06,995,Sarah,0.1556366526089783,-0.5292167458729122
2000-01-01 00:00:07,985,Ingrid,0.36348577151251593,-0.4554382621362476
2000-01-01 00:00:08,965,Zelda,-0.2505504327991166,0.8975113720560848


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,999,Jerry,0.11470716841628548,0.9837349167261586
2000-01-30 00:00:01,1067,Frank,0.5309453412279468,0.17742988765671108
2000-01-30 00:00:02,982,George,0.7443297197151555,0.48776215911504917
2000-01-30 00:00:03,1013,Ray,-0.12579552530413962,0.19386449347497225
2000-01-30 00:00:04,1027,Ursula,-0.7770696825646313,-0.09469926608287738
2000-01-30 00:00:05,1002,Bob,0.837779686153413,0.09292635549818118
2000-01-30 00:00:06,993,Yvonne,-0.2746032925759956,0.9746732081367779
2000-01-30 00:00:07,1033,Victor,0.2216996063257084,0.021026978236116767
2000-01-30 00:00:08,1057,Oliver,-0.010850216271688584,0.07159542621820458


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1015,Yvonne,-0.172694,0.618202
1,2000-01-01 00:00:01,972,Ingrid,0.077038,-0.456993
2,2000-01-01 00:00:02,969,Zelda,0.752608,-0.789773
3,2000-01-01 00:00:03,989,George,-0.830926,0.838535
4,2000-01-01 00:00:04,1016,Tim,-0.964300,0.195447


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1015,Yvonne,-0.172694,0.618202
1,2000-01-01 00:00:01,972,Ingrid,0.077038,-0.456993
2,2000-01-01 00:00:02,969,Zelda,0.752608,-0.789773
3,2000-01-01 00:00:03,989,George,-0.830926,0.838535
4,2000-01-01 00:00:04,1016,Tim,-0.964300,0.195447


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.8 s, sys: 607 ms, total: 5.41 s
Wall time: 3.56 s


name
Alice      -0.000121
Bob        -0.000449
Charlie    -0.001186
Dan        -0.001778
Edith       0.001948
Frank       0.000573
George     -0.000898
Hannah      0.000737
Ingrid      0.004540
Jerry      -0.004989
Kevin      -0.001027
Laura      -0.001540
Michael    -0.000888
Norbert    -0.000773
Oliver      0.001608
Patricia   -0.003549
Quinn       0.000278
Ray         0.000870
Sarah      -0.001142
Tim        -0.002020
Ursula     -0.000367
Victor     -0.000831
Wendy      -0.000694
Xavier      0.002933
Yvonne      0.000876
Zelda       0.001674
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.54 s, sys: 164 ms, total: 1.71 s
Wall time: 1.16 s


name
Alice      -0.000121
Bob        -0.000449
Charlie    -0.001186
Dan        -0.001778
Edith       0.001948
Frank       0.000573
George     -0.000898
Hannah      0.000737
Ingrid      0.004540
Jerry      -0.004989
Kevin      -0.001027
Laura      -0.001540
Michael    -0.000888
Norbert    -0.000773
Oliver      0.001608
Patricia   -0.003549
Quinn       0.000278
Ray         0.000870
Sarah      -0.001142
Tim        -0.002020
Ursula     -0.000367
Victor     -0.000831
Wendy      -0.000694
Xavier      0.002933
Yvonne      0.000876
Zelda       0.001674
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.39 s, sys: 64.5 ms, total: 1.45 s
Wall time: 1.02 s


name
Alice      -0.000121
Bob        -0.000449
Charlie    -0.001186
Dan        -0.001778
Edith       0.001948
Frank       0.000573
George     -0.000898
Hannah      0.000737
Ingrid      0.004540
Jerry      -0.004989
Kevin      -0.001027
Laura      -0.001540
Michael    -0.000888
Norbert    -0.000773
Oliver      0.001608
Patricia   -0.003549
Quinn       0.000278
Ray         0.000870
Sarah      -0.001142
Tim        -0.002020
Ursula     -0.000367
Victor     -0.000831
Wendy      -0.000694
Xavier      0.002933
Yvonne      0.000876
Zelda       0.001674
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html